In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('Social_Network_Ads.csv')

In [3]:
data=pd.get_dummies(data,drop_first=True)
data.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
data['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [6]:
independent=data[['Age', 'EstimatedSalary','Gender_Male']]
dependent=data['Purchased']

In [9]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)


In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid={'penalty':['l2'],'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']}

grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [17]:
y_pred=grid.predict(X_test)

re=grid.cv_results_

table=pd.DataFrame.from_dict(re)
print(f'grid result is',table)


from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(f'cm is',cm)


from sklearn.metrics import classification_report
cf_model=classification_report(y_test,y_pred)
print(f'CF_MODEL is',cf_model)

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print(f'F1 macro of the parameeter value:',grid.best_params_,'F1macro value:',f1_macro)


from sklearn.metrics import roc_auc_score
auc_score=roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

print(f'Auc_score is:',auc_score)

grid result is    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_penalty  \
0       0.009769      0.001475         0.008628        0.001753            l2   
1       0.002941      0.001751         0.007981        0.001515            l2   
2       0.013667      0.004794         0.008989        0.002018            l2   
3       0.015635      0.007702         0.002588        0.002365            l2   
4       0.005604      0.004992         0.003764        0.005681            l2   
5       0.005983      0.006112         0.003632        0.005642            l2   

      param_solver                                          params  \
0            lbfgs            {'penalty': 'l2', 'solver': 'lbfgs'}   
1        liblinear        {'penalty': 'l2', 'solver': 'liblinear'}   
2        newton-cg        {'penalty': 'l2', 'solver': 'newton-cg'}   
3  newton-cholesky  {'penalty': 'l2', 'solver': 'newton-cholesky'}   
4              sag              {'penalty': 'l2', 'solver': 'sag'} 